<a href="https://colab.research.google.com/github/honggilgim/python_test_with_colab/blob/main/nn_%EC%98%88%EC%8B%9C%EC%BD%94%EB%93%9C_%EB%94%B0%EB%9D%BC%ED%95%B4%EB%B3%B4%EA%B8%B0_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
train = pd.read_csv('/content/train2.csv')
test = pd.read_csv('/content/test2.csv')

train.drop('id', axis=1, inplace=True)
test.drop('id', axis=1, inplace=True)

train_X = train.drop('Target', axis=1)
train_y = train.Target

In [3]:
train_X = pd.get_dummies(data = train_X, columns = ['Gender'], prefix = 'Gender')
test = pd.get_dummies(data = test, columns = ['Gender'], prefix = 'Gender')

In [4]:
foreign_body = train_X['Whole Weight'] - (train_X['Shucked Weight'] + train_X['Viscra Weight'] + train_X['Shell Weight'])

In [5]:
foreign_body[47]

0.10650000000000004

In [6]:
area = 0.5 * train_X['Lenght'] * 0.5 * train_X['Diameter'] * np.pi
train_X['Area'] = area

In [7]:
area = 0.5 * test['Lenght'] * 0.5 * test['Diameter'] * np.pi
test['Area'] = area

In [8]:
perimeter = np.pi * np.sqrt(0.5 * ((train_X['Lenght'] ** 2) + (train_X['Diameter'] ** 2)))
train_X['Perimeter'] = perimeter

In [9]:
perimeter = np.pi * np.sqrt(0.5 * ((test['Lenght'] ** 2) + (test['Diameter'] ** 2)))
test['Perimeter'] = perimeter

In [10]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *

In [11]:
# 딥러닝 모델 선언
model = Sequential()
model.add(Dense(16, input_dim=len(train_X.columns), activation='elu'))
model.add(Dense(32, activation='elu'))    
model.add(Dense(64, activation='elu'))  
model.add(Dropout(0.5))  
model.add(Dense(32, activation='elu'))
model.add(Dense(16, activation='elu'))
model.add(Dense(1))


model.compile(loss='mean_absolute_error',
              optimizer='Nadam', 
              metrics=['mae'])

In [12]:
# 모델 저장 폴더 만들기
MODEL_DIR = './model/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)

modelpath = "./model/{epoch:02d}-{val_loss:.4f}.hdf5"

In [13]:
# 모델 업데이트 및 저장
cp = ModelCheckpoint(filepath=modelpath, monitor='val_mae', verbose=0, save_best_only=True, mode = 'min')

In [14]:

# 학습 자동 중단 설정
es = EarlyStopping(monitor='val_mae', patience=50, mode='min')

rlrp = ReduceLROnPlateau(monitor='val_mae', factor=0.2, patience=40, mode='min')

In [15]:
# 모델 학습
model.fit(train_X, train_y, validation_split=0.3, epochs=1000, batch_size=32, verbose=1, callbacks=[es, cp, rlrp])

Epoch 1/1000
25/25 [==============================] - 2s 12ms/step - loss: 24.0766 - mae: 24.0766 - val_loss: 5.2258 - val_mae: 5.2258 - lr: 0.0010
Epoch 2/1000
25/25 [==============================] - 0s 5ms/step - loss: 10.4672 - mae: 10.4672 - val_loss: 4.3781 - val_mae: 4.3781 - lr: 0.0010
Epoch 3/1000
25/25 [==============================] - 0s 5ms/step - loss: 6.8643 - mae: 6.8643 - val_loss: 4.0656 - val_mae: 4.0656 - lr: 0.0010
Epoch 4/1000
25/25 [==============================] - 0s 5ms/step - loss: 5.4075 - mae: 5.4075 - val_loss: 3.9509 - val_mae: 3.9509 - lr: 0.0010
Epoch 5/1000
25/25 [==============================] - 0s 5ms/step - loss: 4.8392 - mae: 4.8392 - val_loss: 3.6901 - val_mae: 3.6901 - lr: 0.0010
Epoch 6/1000
25/25 [==============================] - 0s 5ms/step - loss: 4.3760 - mae: 4.3760 - val_loss: 3.3646 - val_mae: 3.3646 - lr: 0.0010
Epoch 7/1000
25/25 [==============================] - 0s 5ms/step - loss: 3.8735 - mae: 3.8735 - val_loss: 3.1964 - val_mae: 

In [17]:
#Y_prediction = model.predict(test).flatten()